In [21]:
import pandas as pd
import gmaps
import requests
from config import g_key

In [22]:
city_data_df = pd.read_csv('weather_data/cities.csv')
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Albany,US,2020-08-17 21:00:08,42.60,-73.97,75.99,59,4,6.91
1,1,Chuy,UY,2020-08-17 21:01:18,-33.70,-53.46,49.91,76,0,6.17
2,2,Hammerfest,NO,2020-08-17 21:01:19,70.66,23.68,44.60,93,75,28.86
3,3,Beloha,MG,2020-08-17 20:56:47,-25.17,45.05,64.69,94,27,8.52
4,4,Barrow,US,2020-08-17 20:56:31,71.29,-156.79,46.40,87,75,20.80


In [23]:
# check data types
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [24]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key = g_key)

In [25]:
# heatmap of temps
# assign locations to array of lats and longs
locations = city_data_df[['Lat', 'Lng']]

# assign weights variable to some value, in this case max temp
max_temp = city_data_df['Max Temp'] # an array equal to length of locations[]

# assign figure variable to gmaps.figure() attribute
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.6)

# assign heatmap_layer variable to heatmap_layer attribute and add in locations
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp], 
                                    dissipating=False, max_intensity=300, point_radius=4)
# add heatmap_layer
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [26]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.6)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [27]:
# Heatmap of percent cloudiness
locations = city_data_df[["Lat", "Lng"]]
clouds = city_data_df["Cloudiness"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.6)
heat_layer = gmaps.heatmap_layer(locations, weights=clouds, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [28]:
# Heatmap of wind speed
locations = city_data_df[["Lat", "Lng"]]
wind = city_data_df["Wind Speed"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.6)
heat_layer = gmaps.heatmap_layer(locations, weights=wind, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [29]:
# prompt user to add a min and max temp
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [30]:
# filter dataset to find cities that fit temp requirement
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Albany,US,2020-08-17 21:00:08,42.60,-73.97,75.99,59,4,6.91
6,6,San Jose Del Boqueron,HN,2020-08-17 21:01:20,15.48,-87.87,89.60,55,75,11.41
7,7,Bethel,US,2020-08-17 20:58:32,41.37,-73.41,78.01,74,54,4.81
12,12,Honavar,IN,2020-08-17 21:01:21,14.28,74.45,78.66,86,100,7.14
18,18,Hasaki,JP,2020-08-17 21:01:23,35.73,140.83,80.60,100,75,3.36
19,19,Victoria,HK,2020-08-17 20:57:05,22.29,114.16,80.01,83,100,6.24
20,20,Hilo,US,2020-08-17 21:00:29,19.73,-155.09,82.40,58,75,6.93
30,30,Jamestown,US,2020-08-17 21:00:02,42.10,-79.24,75.00,56,1,9.17
36,36,Westport,US,2020-08-17 21:01:04,41.14,-73.36,79.00,73,27,8.90
42,42,Nicoya,CR,2020-08-17 21:01:28,10.15,-85.45,87.80,62,40,4.70


In [31]:
# check for null values
preferred_cities_df.count()

City_ID       199
City          199
Country       199
Date          199
Lat           199
Lng           199
Max Temp      199
Humidity      199
Cloudiness    199
Wind Speed    199
dtype: int64

In [33]:
# new dataframe to store hotels
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Albany,US,75.99,42.60,-73.97,
6,San Jose Del Boqueron,HN,89.60,15.48,-87.87,
7,Bethel,US,78.01,41.37,-73.41,
12,Honavar,IN,78.66,14.28,74.45,
18,Hasaki,JP,80.60,35.73,140.83,
19,Victoria,HK,80.01,22.29,114.16,
20,Hilo,US,82.40,19.73,-155.09,
30,Jamestown,US,75.00,42.10,-79.24,
36,Westport,US,79.00,41.14,-73.36,
42,Nicoya,CR,87.80,10.15,-85.45,


In [34]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [36]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print('Hotel not found... skipping.')

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [37]:
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Albany,US,75.99,42.60,-73.97,
6,San Jose Del Boqueron,HN,89.60,15.48,-87.87,Tienda Nuevo San Juan Tropigas
7,Bethel,US,78.01,41.37,-73.41,Courtyard by Marriott Danbury
12,Honavar,IN,78.66,14.28,74.45,Hotel Sagar Residency
18,Hasaki,JP,80.60,35.73,140.83,Inubosaki Kanko Hotel
19,Victoria,HK,80.01,22.29,114.16,Mini Hotel Central
20,Hilo,US,82.40,19.73,-155.09,Hilo Hawaiian Hotel
30,Jamestown,US,75.00,42.10,-79.24,Hampton Inn & Suites Jamestown
36,Westport,US,79.00,41.14,-73.36,The Westport Inn
42,Nicoya,CR,87.80,10.15,-85.45,Guayacan Real


In [39]:
# heatmap with max_temp from hotels df
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)

fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [41]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [42]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))